In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pydicom
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import os.path
import skimage
import skimage.color
import skimage.transform
import skimage.segmentation
import sklearn.preprocessing
import sklearn.model_selection
import math
import shutil
import pathlib
import glob
import shutil
import uuid
import random
import platform
import torch
import torchvision
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
import pandas as pd
import networkx
import wfdb
import json
import tqdm
import dill
import pickle
import matplotlib.pyplot as plt

import SimpleITK as sitk

import sak
import sak.hash
import sak.signal.wavelet
import sak.data
import sak.data.augmentation
import sak.visualization
import sak.visualization.plot
import sak.torch
import sak.torch.nn
import sak.torch.nn as nn
import sak.torch.train
import sak.torch.data
import sak.data.preprocessing
import sak.torch.models
import sak.torch.models.lego
import sak.torch.models.variational
import sak.torch.models.classification


In [3]:
import xml.etree.ElementTree as ET

In [4]:
ET.register_namespace("","http://www3.medical.philips.com")

In [128]:
files = glob.glob('/home/guille/Descargas/ECGs/*/*.xml')
head = "{http://www3.medical.philips.com}"
namespace = {
    "xmlns:ns0": "http://www3.medical.philips.com",
    "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
    "xsi:schemaLocation": "http://www3.medical.philips.com PhilipsECG.xsd"
}

for file in tqdm.tqdm(files):
    # Load SVG as xml file
    tree = ET.parse(file)

    # Get tree's root (all appended to it)
    root = tree.getroot()

    # Get patient
    patient = root.find(f"{head}patient")

    # Get patient's general data
    generaldata = patient.find(f"{head}generalpatientdata")

    # Modify patient's ID
    generaldata.find(f"{head}patientid").text = "999999"

    # Modify patient's name and surname
    generaldata.find(f"{head}name").find(f"{head}lastname").text = "Anon"
    generaldata.find(f"{head}name").find(f"{head}firstname").text = "Anon"
    generaldata.find(f"{head}name").find(f"{head}middlename").text = "Anon"

    tree.write(file, encoding='utf-16', xml_declaration=True)

100%|██████████| 153/153 [00:01<00:00, 85.74it/s]


In [129]:
file

'/home/guille/Descargas/ECGs/HCM patients/CPH-MCH84.xml'

In [14]:
files = glob.glob('/home/guille/Descargas/ECGs/*/*.xml')
head = "{http://www3.medical.philips.com}"
namespace = {
    "xmlns:ns0": "http://www3.medical.philips.com",
    "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
    "xsi:schemaLocation": "http://www3.medical.philips.com PhilipsECG.xsd"
}

samplingrates = []

for file in tqdm.tqdm(files):
    # Load SVG as xml file
    tree = ET.parse(file)

    # Get tree's root (all appended to it)
    root = tree.getroot()

    # Get patient
    patient = root.find(f"{head}patient")

    # Get Fs
    fs = root.find(f"{head}dataacquisition").find(f"{head}signalcharacteristics").find(f"{head}samplingrate").text
    
    # Store
    samplingrates.append(int(fs))


100%|██████████| 153/153 [00:01<00:00, 138.15it/s]
